In [1]:
# to save import results between script runs
import pyarrow.feather as feather

# Pandas and numpy for data manipulation
import pandas as pd
from pandas import Interval
import numpy as np

import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import iplot

In [2]:
# set pandas visualization options
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_column',None)

# Ensure Orca server is running 
plotly.io.orca.ensure_server()

In [3]:
#receives the dataframe input column and the quantity of quantiles
def find_quantiles_intervals(df_col, quantiles):    
        quant=np.quantile(np.arange(0,len(df_col)), 1/quantiles)
        df_col = df_col.sort_values(inplace=False)
        left=0
        intervals=list()
        for i in range(1,10):
            #display('computing quantile: '+str(i))
            index = int(quant*i)
            right=index
            offset=1
            if (df_col.iloc[right]!=df_col.iloc[right+offset] and 
                  df_col.iloc[right]!=df_col.iloc[right-offset]):
                intervals.append(pd.Interval(left=df_col[left], right=df_col[right]))
                left=right
            else:   
                while (df_col.iloc[right]==df_col.iloc[right+offset] and 
                      df_col.iloc[right]==df_col.iloc[right-offset]):
                      offset=offset+1
                if df_col.iloc[right]!=df_col.iloc[right+offset]:
                    intervals.append(pd.Interval(left=df_col.iloc[left], right=df_col.iloc[right+offset]))
                    left=right+offset
                else:
                    intervals.append(pd.Interval(left=df_col.iloc[left], right=df_col.iloc[right-offset]))
                    left=right-offset
        if right < len(df_col)-1:
            lastInterval = intervals[-1]
            intervals.append(pd.Interval(left=lastInterval.right, right= df_col.max()))    
        display('intervals:')
        display(intervals)


In [4]:
#import data and merge
cod_salmonella =['AASE', 'ABSAL', 'ABSAN', 'ABSEN', 'ABSEV', 'CARSALM', 'ELFSACA', 'ELFSALG', 'ELFSALM', 'ELFSALT', 'ELFSANA', 'ELFSATA', 'ELFSATN', 'ELFSGNA', 'ELFSTNA', 'ELSAL', 'FTIPS', 'FTPS2', 'LMPCR', 'PCDSAL', 'PCRSAL', 'PFGESAL', 'PFSALCR', 'RCSALQL', 'RCSALQN', 'SAFSISC', 'SALDIS', 'SALFSIS', 'SALG2NA', 'SALGUNA', 'SALGUS2', 'SALGUSC', 'SALI2GN', 'SALI2TN', 'SALICA', 'SALICAC', 'SALICN', 'SALICNC', 'SALIS2N', 'SALISC2', 'SALISCN', 'SALISG2', 'SALISGN', 'SALISO', 'SALISO2', 'SALISOC', 'SALISOD', 'SALISOG', 'SALISON', 'SALISOT', 'SALIST2', 'SALISTA', 'SALISTC', 'SALISTN', 'SALITCN', 'SALITN', 'SALM', 'SALM2', 'SALM2NA', 'SALMCON', 'SALMEL', 'SALMNA', 'SALMPCR', 'SALMTA', 'SALPCA', 'SALPCCA', 'SALPCNA', 'SALPCNN', 'SALPCR', 'SALPCR2', 'SALPCRA', 'SALPCRG', 'SALPCRT', 'SALPCTA', 'SALPCTN', 'SALPGNA', 'SALPTNA', 'SALRAP', 'SALSA', 'SALSEL', 'SALSP', 'SALSPAN', 'SALSPAR', 'SALSPCR', 'SALSPNA', 'SALSPR', 'SALSPRR', 'SALT2NA', 'SALTUNA', 'SALTUO', 'SALTUO2', 'SALVPCR', 'SAMELNA', 'SARAPNA', 'SARSPUL', 'TIPSAL2', 'TIPSALM', 'TIPSALP', 'TIPSCR', 'TIPSCR2', 'TIPSISS']
cod_epatite =['ELHA', 'ELHB', 'ELHC', 'ELHE', 'ELHE1', 'ELHEM', 'HACONCR', 'HASCRCR', 'HAVALCR', 'HAVCPCR', 'HAVPCR', 'HAVSPCN', 'HAVSPCR', 'HEVPCR', 'HEVPCR2', 'PCRHAV', 'PCRHAV1', 'PCRHCV', 'PCRHEV', 'PCRHEV1', 'PCRHEVP', 'PEGEPCR', 'RRTHACR', 'RRTHAV', 'RRTHAV2', 'RRTHAV3', 'RRTHAVE', 'RRTHAVN', 'RRTHEV', 'RRTNV', 'RRTNV2', 'SEQHBVP', 'SEQHEV']
cod_leishmania=['IFILEI', 'IFILEIS', 'IFILENA', 'QPCRLEI']
cod_leptospira=['LPAG', 'LPBOC', 'LPCS', 'LPEQ', 'LPEX', 'LPFE', 'LPSEL', 'PCRLPT']

dataframe_towns_geodata=feather.read_feather('dataframe_towns_geodata')
dataframe_sigla=feather.read_feather('dataframe_sigla_water_dist_2_5')#('dataframe_sigla_water_PLV')
df_sigla_w_siiscover=dataframe_sigla.merge(dataframe_towns_geodata, left_on='COD ISTAT COMUNE PRELIEVO', right_on='id comune', how='left')
df_sigla_w_siiscover.head(5)

,RAGIONE SOCATTPRELIEVO,INDIRIZZO ATTPREL,COMUNE ATTPREL,RAGIONE SOCATTPRELCORR,ASL ATTPRELCORRELATA,INDIRIZZO ATTPREL CORRELATA,COMUNE ATTPREL CORRELATA,RAGSOCATTPROVCORR,COD AZ ATT PROV CORR,CODINTERNO ATTPROVCORR,INDIRIZZO ATTPROV CORRELATA,COMUNE ATTPROV,QUESITO DIAGN,PARAMETRO AL CONFERIMENTO,ESAME,RICERCA,SPECIE,TIPO ANIMALE,RAZZA,NACCETTAZIONE,DATA ACCETTAZIONE,NCONFERIMENTO,NAZIONE,ANALISI GARANTITA,NUMVERBALE,DATA PRELIEVO,NCAMPIONE,NOME CAPO,IDENT. CAMPIONE,SEDE ACC,ESITO,aliquota,unita' campionaria,RDP,DATA EMISSIONE RdP,SOSTITUISCE IL RDP N,AnnoRDP sost,anno,cod istat sigla,CAP ATTPROV,PROVINCIA ATTPROV,COD ISTAT COMUNE ATTPROV,cod istat,CAP ATTPREL CORRELATA,PROVINCIA ATTPREL CORRELATA,COD ISTAT COMUNE ATTPREL CORRELATA,INDIRIZZO PRELIEVO,CAP PRELIEVO,PROVINCIA PRELIEVO,COMUNE PRELIEVO,COD ISTAT COMUNE PRELIEVO,id,RDP id,SOSTITUISCE IL RDP id,MESE PRELIEVO,ANNO PRELIEVO,MESE ANNO PRELIEVO,min_dist,count_water_sources,geocode_address,latitude,longitude,id comune,Nome Comune,Nome Regione,id regione,Provincia,sigla,Superficie,Popolazione,altitudine,Grado di urbanizzazione,SHAPE_LENG,Shape_Le_1,Shape_Area,lng,lat,Superfici artificiali e costruzioni (ha),Superfici naturali non vegetate (ha),Alberi (ha),Arbusti (ha),Vegetazione erbacea (ha),Acque e zone umide (ha),Superfici artificiali e costruzioni (%),Superfici naturali non vegetate (%),Alberi (%),Arbusti (%),Vegetazione erbacea (%),Acque e zone umide (%)
0,ASSOCIAZIONE TUTELA E RECUPERO ANIMALI ABBANDO...,LOCALITÀ CASA SOTTO 5,CASTANA,ASSOCIAZIONE TUTELA E RECUPERO ANIMALI ABBANDO...,C328,LOCALITÀ CASA SOTTO 5,CASTANA,<NA>,<NA>,<NA>,<NA>,None,<NA>,<NA>,QPCRLEI,<NA>,CANE,<NA>,<NA>,78910,05/10/2015,1,<NA>,N,-,2015-09-19,2,<NA>,ELVIS,TO,P,0,0,161824,18/12/2015,<NA>,<NA>,"2,015.00",None,<NA>,None,NaN,18036,27040,PV,18036,LOCALITÀ CASA SOTTO 5,27040,PV,CASTANA,18036,QPCRLEI78910--1-2-0-0,1618242015.0,<NA>,9,2015,201509,223.09,193,"LOCALITÀ CASA SOTTO 5,27040,CASTANA,PV,Italia",45.03,9.27,18036,Castana,Lombardia,3,Pavia,PV,5.28,744.00,290.00,3,17459.14017460000,17459.00926490000,5277182.78980000038,9.27,45.03,47.40,0.00,161.27,266.71,52.68,0.00,8.98,0.00,30.54,50.51,9.98,0.00
1,PRIMAVERA SAS DI BADELLINO MARIO E C.,VIA VECCHIA DI PISCINA 2,FROSSASCO,PRIMAVERA SAS DI BADELLINO MARIO E C.,A203,VIA VECCHIA DI PISCINA 2,FROSSASCO,----BOSIO CESARE - CAVOUR,070TO412,2,PASCHERE C.NA PASCARETTA,CAVOUR,ISP,MSU,SALPCNA,<NA>,BOV,<NA>,<NA>,73816,16/09/2015,1,<NA>,N,203/PR,2015-09-15,1,<NA>,<NA>,TO,N,0,0,115920,22/09/2015,<NA>,<NA>,"2,015.00",070TO,10061,TO,"1,070.00",1110,10060,TO,1110,PASCHERE C.NA PASCARETTA,10061,TO,CAVOUR,1070,SALPCNA73816-MSU-1-1-0-0,1159202015.0,<NA>,9,2015,201509,151.54,45,"PASCHERE C.NA PASCARETTA,10061,CAVOUR,TO,Italia",44.77,7.41,1070,Cavour,Piemonte,1,Torino,TO,48.96,"5,568.00",300.00,3,42192.86984060000,42192.54952040001,48959115.12240000069,7.38,44.78,367.43,122.46,466.08,0.16,"3,941.73",0.00,7.50,2.50,9.52,0.00,80.48,0.00
2,AGLIETTO NATURA SRL,VIALE ROMA 9,BIANZE',AGLIETTO NATURA SRL,A206,VIALE ROMA 9,BIANZE',<NA>,<NA>,<NA>,<NA>,None,SPRI,<NA>,SALSP,<NA>,POL,<NA>,<NA>,47095,22/05/2015,2,<NA>,N,-,2015-05-21,6,<NA>,<NA>,TO,N,0,0,72413,26/05/2015,<NA>,<NA>,"2,015.00",None,<NA>,None,NaN,2011,13041,VC,2011,VIALE ROMA 9,13041,VC,BIANZE',2011,SALSP47095--2-6-0-0,724132015.0,<NA>,5,2015,201505,986.23,26,"VIALE ROMA 9,13041,BIANZE',VC,Italia",45.31,8.12,2011,Bianzè,Piemonte,1,Vercelli,VC,41.81,"2,028.00",182.00,3,37036.29780100000,37036.02028130000,41805752.16920000315,8.12,45.31,217.92,0.00,73.27,0.00,"3,891.21",1.04,5.21,0.00,1.75,0.00,93.01,0.02
3,PONTINVREA - TERRITORIO COMUNALE,<NA>,PONTINVREA,PONTINVREA - TERRITORIO COMUNALE,H102,<NA>,PONTINVREA,<NA>,<NA>,<NA>,<NA>,None,CRL,<NA>,IFILEIS,<NA>,CANE,<NA>,<NA>,5176,21/01/2015,1,<NA>,N,S/009/2015,2015-01-20,1,380260041444970,<NA>,SV,N,0,0,11518,29/01/2015,<NA>,<NA>,"2,015.00",None,<NA>,None,NaN,9051,17042,SV,9051,<NA>,17042,SV,PONTINVREA,9051,IFILEIS5176--1-1-0-0,115182015.0,<NA>,1,2015,201501,128.31,8,"17042,PO

In [5]:
y=[ len(df_sigla_w_siiscover[ df_sigla_w_siiscover['ESAME'].isin(cod_leishmania)]['id comune'].unique()), 
          len(df_sigla_w_siiscover[ df_sigla_w_siiscover['ESAME'].isin(cod_leptospira)]['id comune'].unique()), 
          len(df_sigla_w_siiscover[ df_sigla_w_siiscover['ESAME'].isin(cod_salmonella)]['id comune'].unique()), 
          len(df_sigla_w_siiscover[ df_sigla_w_siiscover['ESAME'].isin(cod_epatite)]['id comune'].unique()), 
          len(df_sigla_w_siiscover[ df_sigla_w_siiscover['ESAME'].isin(cod_leishmania+cod_leptospira+cod_salmonella+cod_epatite)]['id comune'].unique())]
fig=px.bar( x=['Leishmaniosis','Leptospirosis', 'Salmonellosis', 'Hepatitis', 'Total'], 
           y=y,
           text=y,   
           title="Quantity of unique Towns referenced by Observations for each zoonose"
          )
fig=fig.update_yaxes(title='', visible=True, showticklabels=True)
fig=fig.update_xaxes(title='', visible=True, showticklabels=True)
fig.write_image("images/SiIsCover_town_distribution_overzoonoses_chart.png")
fig

In [6]:
create_ds_gwr = False #variable for create feather for GWR model
print_graph = False #variable for print correct value of min_dist, only for graph

features_columns=['COD ISTAT COMUNE PRELIEVO',
                           'Superficie', 'Popolazione','altitudine','Superfici artificiali e costruzioni (ha)',
                           'Superfici naturali non vegetate (ha)', 'Alberi (ha)', 'Arbusti (ha)',
                           'Vegetazione erbacea (ha)', 'Acque e zone umide (ha)',
                           'Superfici artificiali e costruzioni (%)','Superfici naturali non vegetate (%)', 
                           'Alberi (%)', 'Arbusti (%)','Vegetazione erbacea (%)', 'Acque e zone umide (%)',
                           'min_dist', 'count_water_sources'
                            ]

coord_columns=['latitude','longitude']

#filter only leishmania data
df_sigla_w_siiscover_leish=df_sigla_w_siiscover[df_sigla_w_siiscover['ESAME'].isin(cod_salmonella)]
datatasets={'Leishmaniasis':df_sigla_w_siiscover[df_sigla_w_siiscover['ESAME'].isin(cod_leishmania)],
            'Leptospirosis':df_sigla_w_siiscover[df_sigla_w_siiscover['ESAME'].isin(cod_leptospira)],
            'Salmonellosis':df_sigla_w_siiscover[df_sigla_w_siiscover['ESAME'].isin(cod_salmonella)],
            'Hepatitis':df_sigla_w_siiscover[df_sigla_w_siiscover['ESAME'].isin(cod_epatite)],}

for label_zoo, data_zoo in datatasets.items():
    first_round = True
    '''
    #(test) two code rows are for check one case of duplicated address and analyse its details. After retrive last exam
    test = data_zoo.loc[(np.isclose(data_zoo['latitude'], 44.216910, rtol=.0000001)) & (np.isclose(data_zoo['longitude'], 8.326127, rtol=.0000001))]
    display(test)
    print(data_zoo.loc[[12]]['latitude'].values)
    print(data_zoo.loc[[11]]['longitude'].values)
    print(data_zoo.loc[[4879]]['latitude'].values)
    print(data_zoo.loc[[4879]]['longitude'].values)
    '''
    data_zoo_copy = data_zoo
    already_checked_idx = {}
    index_res = []
    if create_ds_gwr:
        for idx,row in data_zoo_copy.iterrows():
            if idx not in already_checked_idx.keys():
                if ~np.isnan(row['latitude']) or ~np.isnan(row['longitude']):
                    repeted_lat_long = data_zoo_copy.loc[(np.isclose(data_zoo_copy['latitude'], row['latitude'], rtol=.0000001)) & (np.isclose(data_zoo_copy['longitude'], row['longitude'], rtol=.0000001))]
                    last_exam = repeted_lat_long.sort_values('DATA PRELIEVO').tail(1)
                    index_res.append(last_exam.index[0])
                    for checked_id in repeted_lat_long.index.values:
                        already_checked_idx[checked_id] = 'checked'
                    if first_round:
                        first_round = False
                        repeted_lat_long.to_csv("test_repeated_"+label_zoo+".csv", index=True)
                        last_exam.to_csv("test_last_exam_"+label_zoo+".csv", index=True)
        not_repeted_df = data_zoo_copy.loc[index_res]
        data_zoo = not_repeted_df
    
    if print_graph:    
        data_zoo["min_dist"].replace({-1: 0}, inplace=True)
        data_zoo["count_water_sources"].replace({-1: 0}, inplace=True)
    
    data_zoo=data_zoo[features_columns+coord_columns+['ESITO']]
    data_zoo=data_zoo.set_index('COD ISTAT COMUNE PRELIEVO').reset_index()
    data_zoo.columns=data_zoo.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

    data_zoo["ESITO"].replace({"P": 1, "N": 0}, inplace=True)

    data_zoo['COD_ISTAT_COMUNE_PRELIEVO']=data_zoo['COD_ISTAT_COMUNE_PRELIEVO'].astype(str)
    display(label_zoo+":")
    display(data_zoo.head(5))
    datatasets[label_zoo]=data_zoo

C:\Users\manue\AppData\Local\Temp\ipykernel_31872\525488559.py:58: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



'Leishmaniasis:'

,COD_ISTAT_COMUNE_PRELIEVO,Superficie,Popolazione,altitudine,Superfici_artificiali_e_costruzioni_ha,Superfici_naturali_non_vegetate_ha,Alberi_ha,Arbusti_ha,Vegetazione_erbacea_ha,Acque_e_zone_umide_ha,Superfici_artificiali_e_costruzioni_%,Superfici_naturali_non_vegetate_%,Alberi_%,Arbusti_%,Vegetazione_erbacea_%,Acque_e_zone_umide_%,min_dist,count_water_sources,latitude,longitude,ESITO
0,18036,5.28,744.00,290.00,47.40,0.00,161.27,266.71,52.68,0.00,8.98,0.00,30.54,50.51,9.98,0.00,223.09,193,45.03,9.27,1
1,9051,24.95,846.00,425.00,114.40,0.00,"2,321.54",9.24,51.17,0.00,4.58,0.00,93.00,0.37,2.05,0.00,128.31,8,44.44,8.43,0
2,8050,2.07,"2,861.00",2.00,82.83,0.00,22.83,0.00,100.48,1.19,39.95,0.00,11.01,0.00,48.46,0.57,67.24,19,43.84,7.88,0
3,6172,8.80,907.00,222.00,86.73,0.00,348.54,107.29,316.30,21.45,9.85,0.00,39.59,12.19,35.93,2.44,381.89,425,44.67,8.43,0
4,8031,45.38,"42,322.00",10.00,"1,015.84",21.72,"2,912.42",163.49,413.99,13.61,22.37,0.48,64.14,3.60,9.12,0.30,212.42,16,43.90,8.03,0


C:\Users\manue\AppData\Local\Temp\ipykernel_31872\525488559.py:58: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



'Leptospirosis:'

...

'Salmonellosis:'

,COD_ISTAT_COMUNE_PRELIEVO,Superficie,Popolazione,altitudine,Superfici_artificiali_e_costruzioni_ha,Superfici_naturali_non_vegetate_ha,Alberi_ha,Arbusti_ha,Vegetazione_erbacea_ha,Acque_e_zone_umide_ha,Superfici_artificiali_e_costruzioni_%,Superfici_naturali_non_vegetate_%,Alberi_%,Arbusti_%,Vegetazione_erbacea_%,Acque_e_zone_umide_%,min_dist,count_water_sources,latitude,longitude,ESITO
0,1070,48.96,"5,568.00",300.00,367.43,122.46,466.08,0.16,"3,941.73",0.00,7.50,2.50,9.52,0.00,80.48,0.00,151.54,45,44.77,7.41,0
1,2011,41.81,"2,028.00",182.00,217.92,0.00,73.27,0.00,"3,891.21",1.04,5.21,0.00,1.75,0.00,93.01,0.02,986.23,26,45.31,8.12,0
2,43012,148.20,"10,509.00",631.00,888.45,0.00,"3,775.97",164.37,"9,752.92",207.72,6.01,0.00,25.53,1.11,65.95,1.40,433.31,17,43.40,13.20,1
3,4242,25.74,"3,167.00",604.00,227.59,0.00,"1,317.36",0.00,"1,030.47",0.00,8.84,0.00,51.15,0.00,40.01,0.00,971.45,60,44.37,7.87,0
4,6107,5.45,326.00,197.00,30.69,0.00,209.55,1.63,303.56,0.00,5.63,0.00,38.42,0.30,55.66,0.00,37.07,25,44.83,8.97,0


C:\Users\manue\AppData\Local\Temp\ipykernel_31872\525488559.py:58: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



'Hepatitis:'

,COD_ISTAT_COMUNE_PRELIEVO,Superficie,Popolazione,altitudine,Superfici_artificiali_e_costruzioni_ha,Superfici_naturali_non_vegetate_ha,Alberi_ha,Arbusti_ha,Vegetazione_erbacea_ha,Acque_e_zone_umide_ha,Superfici_artificiali_e_costruzioni_%,Superfici_naturali_non_vegetate_%,Alberi_%,Arbusti_%,Vegetazione_erbacea_%,Acque_e_zone_umide_%,min_dist,count_water_sources,latitude,longitude,ESITO
0,3073,20.64,"3,617.00",241.00,225.02,60.14,"1,158.56",46.04,561.81,14.31,10.89,2.91,56.08,2.23,27.19,0.69,49.97,113,45.60,8.42,0
1,4016,17.69,"3,200.00",491.00,205.96,0.00,613.98,0.00,949.44,0.00,11.64,0.00,34.70,0.00,53.66,0.00,123.04,80,44.36,7.65,0
2,1272,130.06,"872,367.00",239.00,"8,546.60",46.14,"2,516.16",15.07,"1,686.51",202.77,65.68,0.35,19.34,0.12,12.96,1.56,279.78,448,45.09,7.70,0
3,2156,102.97,"7,542.00",450.00,328.23,32.95,"8,944.30",161.75,800.85,36.50,3.19,0.32,86.80,1.57,7.77,0.35,117.41,1307,45.82,8.25,0
4,7003,21.39,"34,102.00",583.00,660.05,0.00,755.45,151.88,571.08,1.33,30.85,0.00,35.30,7.10,26.69,0.06,95.60,59,45.73,7.31,0


In [7]:
features_columns=[col_name.replace(' ', '_')
              .replace('(', '')
              .replace(')', '') 
              for col_name in features_columns]  
features_columns_red= features_columns.copy()
features_columns_red.remove('COD_ISTAT_COMUNE_PRELIEVO')

for zoo_label, zoo_data in datatasets.items():
    for col in features_columns_red:
        data_feature=zoo_data[[col,'ESITO']]
        
        # use predefined bins in case of percentage field
        
        if '%' in col:
             bins= np.linspace(0,100,10)
        # calculate bins in case of real values    
        else:
            bins= np.linspace(data_feature[col].min(),data_feature[col].max(),10)
        
        names= ["({:.0f}".format( bins[i-1])+" - " +"{:.0f}) ".format( bins[i]) for i in range(1,len(bins))]
            
        # create the bins column
        data_feature[col+'_bins'] = pd.cut(data_feature[col], bins, labels=names, include_lowest=True)
        
        #drop the original column so only the bin one remains
        data_feature=data_feature.drop([col], axis=1)
        
        # filter only positive and drop ESITO col
        data_feature_p=data_feature[data_feature['ESITO']==1].drop(['ESITO'], axis=1)
        len_p=len(data_feature_p)
        
        # filter only negarive and drop ESITO col
        data_feature_n=data_feature[data_feature['ESITO']==0].drop(['ESITO'], axis=1)
        len_n=len(data_feature_n)
        
        # group by bins and count only Negative
        data_feature_n=data_feature_n.groupby([col+'_bins']).size().reset_index(name='counts')
        data_feature_n['ratio']=data_feature_n['counts']/(len_n+len_p)*100
        data_feature_n['type']='Neg'
        
        # group by bins and count only Positive
        data_feature_p=data_feature_p.groupby([col+'_bins']).size().reset_index(name='counts')
        data_feature_p['ratio']=data_feature_p['counts']/(len_n+len_p)*100
        data_feature_p['type']='Pos'
        
        # rebuild the dataset
        data_feature_p_n=data_feature_p.append(data_feature_n)

        # group by bins and count total obeservations
        data_feature_tot_np=data_feature.drop(['ESITO'], axis=1)
        data_feature_tot_np=data_feature_tot_np.groupby([col+'_bins']).size().reset_index(name='counts')
        data_feature_tot_np['type']='Total'
        
        fig1=px.line(data_feature_p_n, 
           x=col+'_bins', 
           y="ratio", 
           color="type", 
           #barmode="group"
          )


        fig2=px.bar(data_feature_tot_np, 
           x=col+'_bins', 
           y="counts", 
           color="type", 
          # barmode="group"
          )

        fig = make_subplots(rows=1, cols=2)
        fig.update_layout(
            legend=dict( yanchor="top",y=0.95,xanchor="left",x=0.64),

            #title=zoo_label': '+ col + ' analisys:'
    )
        for d in fig1.data:
            fig.add_trace(go.Scatter(x=d['x'], y=d['y'], name = d['name']), row=1, col=1)
            #fig.update_yaxes(tickprefix="$")
        for d in fig2.data:
            fig.add_trace((go.Bar(x=d['x'], y=d['y'],  name = d['name'])), row=1, col=2, )
        display(fig)
        fig.write_image(('images/geofeaturescharts/SiIsCover_'+ zoo_label +'_feature_'+ col+'_probability_ranges.png').replace('%','percentage'))
        display(len_p)
        display(len_n)
        #display(data_feature.groupby([col+'_bins', 'ESITO']).size().reset_index(name='counts'))

17449

65837

17449

65837

17449

...

9286

1207

9286

1207

9286

In [8]:


#set bins values and names for percentage values
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
names = ['0-10', '10-20', '20-30', '30-40', '40-50','50-60','60-70','70-80','80-90','90-100']

for zoo_label, zoo_data in datatasets.items():
    for col in features_columns_red:
        data_feature=zoo_data[[col,'ESITO']]
        
        # use predefined bins in case of percentage field
        if '%' in col:
            bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
            names = ['0-10', '10-20', '20-30', '30-40', '40-50','50-60','60-70','70-80','80-90','90-100']
        # calculate bins in case of real values    
        else:
            bins= np.linspace(data_feature[col].min(),data_feature[col].max(),100)
            names= ["({:.0f}".format( bins[i-1])+" - " +"{:.0f}) ".format( bins[i]) for i in range(1,len(bins))]
            
        # create the bins column
        #data_feature[col+'_bins'] = pd.cut(data_feature[col], bins, labels=names, include_lowest=True)
        display(zoo_label)
        display('working on '+col)
        data_feature[col+'_bins'] = pd.qcut(data_feature[col], 10, duplicates='drop')
        
        
       
        #display(data_feature.groupby([col+'_bins']).count() )
        ### data_feature Example
        #    Alberi_ha   ESITO   Alberi_ha_bins
        #0     161.27      1       (0 - 282)
        #1   2,321.54      0     (2256 - 2538)
        #2      22.83      0       (0 - 282)
        
        
        #drop the original column so only the bin and 'ESITO' ones remains
        data_feature=data_feature.drop([col], axis=1)
        len_p_n=len(data_feature)
        
        # filter only positive and drop ESITO col
        data_feature_p=data_feature[data_feature['ESITO']==1].drop(['ESITO'], axis=1)
        len_p=len(data_feature_p)
        
        # group by bins and count only Positive
        data_feature_p=data_feature_p.groupby([col+'_bins']).size().reset_index(name='counts')
        data_feature_p['probability']=data_feature_p['counts']/len_p_n*100
        data_feature_p['type']='Pos'
        
        display("data_feature_p['counts']sum:"+ str(data_feature_p['counts'].sum()))
        display("data_feature_p['probability']sum"+ str(data_feature_p['probability'].sum()))


        # group by bins and count total obeservations
        data_feature_tot_np=data_feature.drop(['ESITO'], axis=1)
        data_feature_tot_np=data_feature_tot_np.groupby([col+'_bins']).size().reset_index(name='counts')
        data_feature_tot_np['type']='Total'
        
        data_feature_tot_np.sort_values(by=[col+'_bins'],inplace=True)
    
        # format bins for visualization
        data_feature_tot_np[col+'_bins']=data_feature_tot_np[col+'_bins'].astype(str)

        
        bin_to_split=data_feature_tot_np[col+'_bins'].str.replace('(','').str.replace(']','')
        splitted_bin=pd.DataFrame(bin_to_split)
        splitted_bin=pd.DataFrame( splitted_bin[col+'_bins'].str.split(',').tolist(),columns = ['left','right'])
        splitted_bin['left']= splitted_bin['left'].astype(float).map('{:.2f}'.format)
        splitted_bin['right']= splitted_bin['right'].astype(float).map('{:.2f}'.format)
        data_feature_tot_np[col+'_bins']="("+splitted_bin['left']+' - ' + splitted_bin['right'] + ']'
        # splitted_bin['left','right']= splitted_bin.str.split('-', 1, expand=True)
        display(splitted_bin)
        #splitted_bin[['left','right']]= 
        #display(splitted_bin)
        display('data_feature_tot_np')
        display(data_feature_tot_np)
        
        data_feature_p.sort_values(by=[col+'_bins'],inplace=True)
        data_feature_p[col+'_bins']=data_feature_tot_np[col+'_bins'].astype(str)
        
        display('data_feature_p')
        display(data_feature_p)
        
        fig1=px.line(data_feature_p, 
           x=col+'_bins', 
           y="probability", 
           color="type", 
           #barmode="group"
          )


        fig2=px.bar(data_feature_tot_np, 
           x=col+'_bins', 
           y="counts", 
           color="type", 
          # barmode="group"
          )

        fig = make_subplots(rows=1, cols=2)
        fig.update_layout(
            legend=dict( yanchor="top",y=0.95,xanchor="left",x=0.64),
            #title=zoo_label+': '+ col + ' analisys:'
    )
        for d in fig1.data:
            fig.add_trace(go.Scatter(x=d['x'], y=d['y'], name = d['name']), row=1, col=1)
            #fig.update_yaxes(tickprefix="$")
        for d in fig2.data:
            fig.add_trace((go.Bar(x=d['x'], y=d['y'],  name = d['name'])), row=1, col=2, )
        display(fig)
        fig.write_image(('images/geofeaturescharts/SiIsCover_'+ zoo_label +'_feature_'+ col+'_probability_deciles.png').replace('%','percentage'))
        display("Tot p: "+ str(len_p))
        display("Tot p+n: "+ str(len_p_n))
        display("Tot p prob: "+ str(len_p/len_p_n*100))
        #display(data_feature.groupby([col+'_bins', 'ESITO']).size().reset_index(name='counts'))

'Leishmaniasis'

'working on Superficie'

"data_feature_p['counts']sum:17449"

"data_feature_p['probability']sum20.950699997598637"

C:\Users\manue\AppData\Local\Temp\ipykernel_31872\4061324666.py:62: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



...

'data_feature_p'

,count_water_sources_bins,counts,probability,type
0,(-1.00 - 11.00],84,0.80,Pos
1,(11.00 - 25.00],107,1.02,Pos
2,(25.00 - 33.00],641,6.11,Pos
3,(33.00 - 52.00],78,0.74,Pos
4,(52.00 - 62.00],22,0.21,Pos
5,(62.00 - 82.00],19,0.18,Pos
6,(82.00 - 212.00],61,0.58,Pos
7,(212.00 - 448.00],169,1.61,Pos
8,(448.00 - 1766.00],26,0.25,Pos


'Tot p: 1207'

'Tot p+n: 10493'

'Tot p prob: 11.502906699704566'

In [9]:
for zoo_label, zoo_data in datatasets.items():
    for col in features_columns_red:
        zoo_data[col]=zoo_data[col].fillna(0)
        zoo_data[col] = zoo_data[col].replace({0: zoo_data[col].median()})
    zoo_data.head(5)
    zoo_data = zoo_data.drop(columns='COD_ISTAT_COMUNE_PRELIEVO')
    corrs = zoo_data.corr()
    display(zoo_label+': ')
    display(corrs['ESITO'])
    
    # Correlation Heatmap
    iplot(ff.create_annotated_heatmap(corrs.iloc[:17, :17].round(3).values, x=list(corrs.iloc[:17, :17].columns), 
                                  y=list(corrs.iloc[:17, :17].index), annotation_text=corrs.iloc[:17, :17].round(3).values))

'Leishmaniasis: '

Superficie                               -0.03
Popolazione                              -0.02
altitudine                                0.06
Superfici_artificiali_e_costruzioni_ha   -0.02
Superfici_naturali_non_vegetate_ha       -0.01
Alberi_ha                                -0.07
Arbusti_ha                               -0.00
Vegetazione_erbacea_ha                    0.00
Acque_e_zone_umide_ha                    -0.00
Superfici_artificiali_e_costruzioni_%     0.00
Superfici_naturali_non_vegetate_%        -0.00
Alberi_%                                 -0.07
Arbusti_%                                 0.08
Vegetazione_erbacea_%                     0.05
Acque_e_zone_umide_%                     -0.00
min_dist                                  0.03
count_water_sources                       0.03
latitude                                  0.01
longitude                                -0.03
ESITO                                     1.00
Name: ESITO, dtype: float64

'Leptospirosis: '

Superficie                                0.06
Popolazione                               0.08
altitudine                                0.02
Superfici_artificiali_e_costruzioni_ha    0.08
Superfici_naturali_non_vegetate_ha        0.05
Alberi_ha                                -0.04
Arbusti_ha                               -0.01
Vegetazione_erbacea_ha                    0.06
Acque_e_zone_umide_ha                     0.06
Superfici_artificiali_e_costruzioni_%     0.05
Superfici_naturali_non_vegetate_%         0.04
Alberi_%                                 -0.08
Arbusti_%                                -0.04
Vegetazione_erbacea_%                     0.03
Acque_e_zone_umide_%                      0.03
min_dist                                 -0.21
count_water_sources                       0.17
latitude                                  0.18
longitude                                -0.05
ESITO                                     1.00
Name: ESITO, dtype: float64

'Salmonellosis: '

Superficie                                0.16
Popolazione                               0.14
altitudine                               -0.10
Superfici_artificiali_e_costruzioni_ha    0.16
Superfici_naturali_non_vegetate_ha       -0.01
Alberi_ha                                 0.12
Arbusti_ha                                0.08
Vegetazione_erbacea_ha                    0.08
Acque_e_zone_umide_ha                     0.02
Superfici_artificiali_e_costruzioni_%     0.16
Superfici_naturali_non_vegetate_%        -0.04
Alberi_%                                 -0.02
Arbusti_%                                -0.00
Vegetazione_erbacea_%                    -0.08
Acque_e_zone_umide_%                      0.06
min_dist                                 -0.03
count_water_sources                       0.09
latitude                                 -0.02
longitude                                 0.01
ESITO                                     1.00
Name: ESITO, dtype: float64

'Hepatitis: '

Superficie                                0.02
Popolazione                               0.07
altitudine                               -0.15
Superfici_artificiali_e_costruzioni_ha    0.08
Superfici_naturali_non_vegetate_ha       -0.08
Alberi_ha                                 0.08
Arbusti_ha                               -0.07
Vegetazione_erbacea_ha                   -0.06
Acque_e_zone_umide_ha                     0.00
Superfici_artificiali_e_costruzioni_%     0.14
Superfici_naturali_non_vegetate_%        -0.09
Alberi_%                                  0.10
Arbusti_%                                -0.10
Vegetazione_erbacea_%                    -0.14
Acque_e_zone_umide_%                     -0.03
min_dist                                  0.07
count_water_sources                      -0.07
latitude                                 -0.20
longitude                                 0.09
ESITO                                     1.00
Name: ESITO, dtype: float64

In [10]:
# distribution of observations by regions

for zoonoose, _ in datatasets.items():
    datatasets_w_regions=datatasets[zoonoose][['COD_ISTAT_COMUNE_PRELIEVO', 'ESITO']]
    datatasets_w_regions['COD_ISTAT_COMUNE_PRELIEVO']=datatasets_w_regions['COD_ISTAT_COMUNE_PRELIEVO'].astype(int)
    datatasets_w_regions=datatasets_w_regions.merge(dataframe_towns_geodata, left_on='COD_ISTAT_COMUNE_PRELIEVO', right_on='id comune', how='left')
    datatasets_w_regions=datatasets_w_regions[['Nome Regione','ESITO']]
    datatasets_w_regions = pd.get_dummies(datatasets_w_regions, columns = ['ESITO']).rename(columns = {'ESITO_0': 'Neg', 'ESITO_1': 'Pos'})
    datatasets_w_regions['Tot']=datatasets_w_regions['Neg']+datatasets_w_regions['Pos']
    datatasets_w_regions=datatasets_w_regions.groupby(['Nome Regione']).agg({'Neg':'sum','Pos':'sum','Tot':'sum'}).reset_index()
    datatasets_w_regions=datatasets_w_regions.sort_values(by='Tot', ascending=False)
    datatasets_w_regions.head(5)

    fig = go.Figure(data=[  
        go.Bar(name='Pos Observations', x=datatasets_w_regions['Nome Regione'], y=datatasets_w_regions['Pos']),
        go.Bar(name='Neg Observations', x=datatasets_w_regions['Nome Regione'], y=datatasets_w_regions['Neg'])
    ])
    # Change the bar mode
    fig.update_layout(barmode='stack',
                     legend=dict( yanchor="top",y=0.95,xanchor="left",x=0.64),)
    fig.write_image(('images/geofeaturescharts/SiIsCover_'+ zoonoose +'_obeservations_distributions_regions.png').replace('%','percentage'))
        

    fig.show()

In [11]:
#make available datasets to other notebooks
for label_zoo, data_zoo in datatasets.items():
    feather_filename = 'SiIsCover_'+label_zoo
    if create_ds_gwr:
        feather_filename += 'GWR'
    feather.write_feather(data_zoo, feather_filename)